Start installing openai:

In [6]:
%pip install openai
import json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 9.2 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0
  Attempting uninstall: rich
    Found existing installation: rich 13.3.2
    Uninstalling rich-13.3.2:
      

Define the prompt to pass to the OpenAI api to perform NER:


In [9]:
PROMPT_TEMPLATE = """
Perform name entity recognition in Spanish. The classes are books, films, videogames, songs, places, dates, topics, organizations and people. The output should follow the format:

[{'class': 'people', 'text': "name of the person"}, {'class': 'books', 'start': 'name of the book}]

Sentence: 

"""


 



Define a function to call the api of OpenAI:


In [10]:
from json import loads
import os
import openai

# set your api key as ENV, for example with Python: os.environ["OPENAI_API_KEY"] = "your api key"
os.environ["OPENAI_API_KEY"] = "put_here_your_key"
openai.api_key = os.getenv("OPENAI_API_KEY") 

def classify(text):
    # build prompt with template and input
    prompt = f"{PROMPT_TEMPLATE}\n{text}\n"
    # use create completion template
    completion = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      temperature=0,
      max_tokens=512
    )
    # get first choice text
    json_response = completion["choices"][0]["text"].strip()

    return json_response

Define the input and output folders and call the OpenAI api:

In [11]:


# specify folder of input transcriptions

transcription_folder = "/content/drive/MyDrive/hackathon_podcast/data/NER/processed_transcription/"

# specify folder of output jsons

json_folder = "/content/drive/MyDrive/hackathon_podcast/data/NER/json_entity/"


import ast

input_file = open(transcription_folder + 'output_0.txt', 'r')

json_list = []

for line in input_file:

  # Process each line of the input file through the OpenAI api
  output = classify(line)

  # The output is a str and we need to reformat as list[Dict]
  try:
    # Transform the string into a list
    output_list = ast.literal_eval(output)
    # Check if the format is a list of dicts, otherwise don't append it
    if isinstance(output_list, list) and all(isinstance(item, dict) for item in output_list):
      json_format = {
          "text": line,
          "entities": ast.literal_eval(output) 
      }
      json_list.append(json_format)

  except:
        # for some examples, json is not correctly formatted
        json_format = {
          "text": line,
          "entities": [] # empty list
        }

input_file.close()

# Store the list of dicts in a json file
with open(json_folder + 'output_0.json', 'w') as f:
    json.dump(json_list, f)


How many samples are there in the json file? What are the entities of the first sentence?

In [22]:
with open(json_folder + 'output_0.json', 'r') as f:
    data = json.load(f)


print(f'There are {len(data)} sentences in the json file')
print(f'The entities of the first sentence are {data[0]["entities"]}')

There are 37 sentences in the json file
The entities of the first sentence are [{'class': 'people', 'text': 'Katie Hesel'}, {'class': 'books', 'text': 'Historia del arte sin hombres'}]
